<a href="https://colab.research.google.com/github/Neerajs004/Plant-Chatbot/blob/main/PlantDoc(New_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Perfrom image augmentation and create training/validation splits**

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
BASE_DIR = '/content/drive/MyDrive/PlantBot'


In [6]:
from google.colab import files
files.upload()     # choose kaggle.json


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"neerajs004","key":"7a1a8679949127c9ad44e3b00284dc2b"}'}

In [7]:
!mkdir -p ~/.kaggle



In [8]:
!cp "kaggle (1) (1).json" ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle (1) (1).json': No such file or directory


In [9]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [10]:
!kaggle datasets download -d emmarex/plantdisease -p /content

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [11]:
!ls /content/plantvillage_data/plantvillage/PlantVillage | head

ls: cannot access '/content/plantvillage_data/plantvillage/PlantVillage': No such file or directory


In [12]:
import os, shutil, random
from glob import glob

src_root = "/content/plantvillage_data/plantvillage/PlantVillage"
dst_root = "/content/plantvillage_split"

train_dir = os.path.join(dst_root, "train")
val_dir = os.path.join(dst_root, "val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

classes = os.listdir(src_root)
split_ratio = 0.8  # 80% train / 20% val

for cls in classes:
    src_cls = os.path.join(src_root, cls)
    images = glob(os.path.join(src_cls, "*.jpg"))

    random.shuffle(images)
    split = int(len(images) * split_ratio)

    train_imgs = images[:split]
    val_imgs = images[split:]

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, cls))

    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, cls))

print("✔ Train/val split completed successfully!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/plantvillage_data/plantvillage/PlantVillage'

In [ ]:

!ls /content/plantvillage_split/train | head

In [ ]:
!ls /content/plantvillage_split/val | head

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_dir = "/content/plantvillage_split/train"
val_dir = "/content/plantvillage_split/val"

train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_data = datasets.ImageFolder(train_dir, transform=train_tfms)
val_data = datasets.ImageFolder(val_dir, transform=val_tfms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
num_classes = len(train_data.classes)
print("Number of disease classes:", num_classes)

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
!find /content/plantvillage_split/train -type f | head

In [ ]:
import os

root = "/content/plantvillage_split/train"

for folder in os.listdir(root):
    old = os.path.join(root, folder)
    new = os.path.join(root, folder.lower())
    if old != new:
        os.rename(old, new)

root = "/content/plantvillage_split/val"

for folder in os.listdir(root):
    old = os.path.join(root, folder)
    new = os.path.join(root, folder.lower())
    if old != new:
        os.rename(old, new)

In [ ]:
!ls /content/plantvillage_split/train

In [ ]:
import os

train_path = "/content/plantvillage_split/train"
val_path = "/content/plantvillage_split/val"

def clean_names(path):
    for folder in os.listdir(path):
        old = os.path.join(path, folder)
        clean = folder.replace("__", "_")  # fix double underscores
        clean = clean.lower().strip()      # enforce lowercase
        new = os.path.join(path, clean)

        if old != new:
            print("RENAMING:", old, "→", new)
            os.rename(old, new)

clean_names(train_path)
clean_names(val_path)

In [ ]:
!ls /content/plantvillage_split/train

In [ ]:
import os
import re

root_paths = [
    "/content/plantvillage_split/train",
    "/content/plantvillage_split/val"
]

def clean_name(name):
    name = name.lower()                     # lowercase
    name = re.sub(r'_+', '_', name)         # replace multiple underscores with 1
    name = name.strip('_')                  # remove leading/trailing underscore
    return name

for root in root_paths:
    for folder in os.listdir(root):
        old = os.path.join(root, folder)
        new_name = clean_name(folder)
        new = os.path.join(root, new_name)

        if old != new:
            print("RENAMING:", folder, "--->", new_name)
            os.rename(old, new)

In [ ]:
!ls /content/plantvillage_split/train

In [ ]:
!ls -1 /content/plantvillage_split/train

In [ ]:
!ls -1 /content/plantvillage_split/train

In [ ]:
!find /content -type d -name "pepper_bell_bacterial_spot"

In [ ]:
train_dir = "THE_REAL_PATH_FROM_OUTPUT"
val_dir   = "THE_REAL_PATH_FROM_OUTPUT/../val"

In [ ]:
!find /content -type d -name "pepper_bell_bacterial_spot"

In [ ]:
!find /content/plantvillage_split/train -type f | wc -l

In [ ]:
!tree /content/plantvillage_split/train | head -n 50

In [ ]:
!ls -R /content/plantvillage_split/train | head -n 50

In [ ]:
!ls /content

In [ ]:
!ls ~/.kaggle

In [ ]:
!rm -rf /content/plantvillage_split

In [ ]:
!ls /content

In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content

In [ ]:
!ls /content

In [ ]:
!unzip -o /content/plantdisease.zip -d /content/plantvillage_raw

In [ ]:
!ls /content/plantvillage_raw

In [ ]:

!ls /content/plantvillage_raw/plantvillage/PlantVillage | head

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

splitfolders.ratio(
    "/content/plantvillage_raw/plantvillage/PlantVillage",
    output="/content/plantvillage_split",
    seed=42,
    ratio=(.8, .2)
)

In [ ]:
!find /content/plantvillage_split/train -type f | wc -l

In [ ]:
!find /content/plantvillage_split/val -type f | wc -l

In [ ]:
train_dir = "/content/plantvillage_split/train"
val_dir   = "/content/plantvillage_split/val"

train_data = datasets.ImageFolder(train_dir, transform=train_tfms)
val_data   = datasets.ImageFolder(val_dir, transform=val_tfms)

print("Train images:", len(train_data))
print("Val images:", len(val_data))
print("Classes:", train_data.classes)

from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=batch_size)

len(train_loader), len(val_loader)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std= [0.229, 0.224, 0.225]
    )
])

SPLIT DATASET (MANDATORY – AGAIN)

In [ ]:
import os, shutil, random

raw_dir = "/content/plantvillage_data/plantvillage/PlantVillage"
train_dir = "/content/data/train"
val_dir = "/content/data/val"

split_ratio = 0.8

for cls in os.listdir(raw_dir):
    cls_path = os.path.join(raw_dir, cls)
    if not os.path.isdir(cls_path):
        continue

    images = os.listdir(cls_path)
    if len(images) == 0:
        continue

    random.shuffle(images)
    split = int(len(images) * split_ratio)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for img in images[:split]:
        shutil.copy(
            os.path.join(cls_path, img),
            os.path.join(train_dir, cls, img)
        )

    for img in images[split:]:
        shutil.copy(
            os.path.join(cls_path, img),
            os.path.join(val_dir, cls, img)
        )

print("✅ Dataset split completed")

VERIFY TRAIN / VAL CONTENTS

In [13]:
import os

train_dir = "/content/data/train"
val_dir = "/content/data/val"

print("Train folder exists:", os.path.exists(train_dir))
print("Val folder exists:", os.path.exists(val_dir))

print("Train classes:", os.listdir(train_dir))
print("Val classes:", os.listdir(val_dir))

Train folder exists: False
Val folder exists: False


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/train'

In [14]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Transforms defined")

Transforms defined


In [15]:
from torchvision import datasets

train_dataset = datasets.ImageFolder(
    root=train_dir,
    transform=train_transform
)

val_dataset = datasets.ImageFolder(
    root=val_dir,
    transform=val_transform
)

print("Train images:", len(train_dataset))
print("Val images:", len(val_dataset))


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/train'

SAVE MODEL + CLASS NAMES

In [16]:
import torch

model_save_path = "/content/plant_disease_resnet50.pth"

torch.save({
    "model_state_dict": model.state_dict(),
    "class_names": train_dataset.classes
}, model_save_path)

print("✅ Model saved successfully at:", model_save_path)

NameError: name 'model' is not defined